In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
#A package for easing return of multiple values
!nvidia-smi
#Create and update Microsoft Word .docx files.
!pip install python-docx
from tensorflow.keras.preprocessing.text import Tokenizer
import docx
import re

Tue May 18 10:29:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [30]:
import tensorflow as tf
import string
import requests

In [21]:
text1 = (open("/content/drive/My Drive/sithara_songs_data.txt").read())
text1=text.lower()

In [26]:
data = text1.split('\n') #split the text with respect to new line
data[0]

'ethu mazhayilum - udaharanam sujatha'

In [27]:
len(data)


1370

The total number of lines in our data is 1370

In [28]:
data = " ".join(data)  #join all the lines to form a long string 
data[:200]

'ethu mazhayilum - udaharanam sujatha  ethu mazhayilaalumoru thirinaalam raavu pakaloru pole viriyana thaaram novu karipukayoothi erivirakaay meenaveyilodu koode vazhi palatheri paadamezhuthiya jeevaka'

In [31]:
def clean_text(data):  #to remove all the punctuation mark and special characters
  tokens = data.split()  #split the data according to space character
  table = str.maketrans('', '', string.punctuation) #delete the punctations ie., the maketrans() has 3 arguments 1st the characters that needs to be replaced with,2 nd 
                                                    # what to replace and 3rd the characters to be deleted.Here, we delete the punctuation
  tokens = [w.translate(table) for w in tokens]
  #The translate() method returns a string where some specified characters are replaced with the character described in a dictionary, or in a mapping table.                                                  
  tokens = [word for word in tokens if word.isalpha()]
  #The isalpha() method returns True if all the characters are alphabet letters (a-z)
  tokens = [word.lower() for word in tokens] #to lower case
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['ethu', 'mazhayilum', 'udaharanam', 'sujatha', 'ethu', 'mazhayilaalumoru', 'thirinaalam', 'raavu', 'pakaloru', 'pole', 'viriyana', 'thaaram', 'novu', 'karipukayoothi', 'erivirakaay', 'meenaveyilodu', 'koode', 'vazhi', 'palatheri', 'paadamezhuthiya', 'jeevakathayude', 'ediloru', 'lipiyaayi', 'ere', 'vijanatha', 'choozhum', 'irulilum', 'aashayathu', 'kalayaathe', 'chiri', 'thooki', 'mukilaayi', 'nee', 'niravaalin', 'kara', 'thedi', 'ethu', 'mazhayilaalumoru', 'thirinaalam', 'raavu', 'pakaloru', 'pole', 'viriyana', 'thaaram', 'hridayamulayum', 'murivu', 'mozhiyaal', 'moodi', 'samayamani', 'pol']


In [32]:
len(tokens)

4030

The total number of words are 4030

In [33]:
len(set(tokens))

2035

The total number of unique words are 2035

In [34]:
length = 10 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  

print(len(lines))

4019


In [35]:
lines[0]

'ethu mazhayilum udaharanam sujatha ethu mazhayilaalumoru thirinaalam raavu pakaloru pole viriyana'

In [36]:
tokens[10]

'viriyana'

In [37]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

sequences contains a list of integer values created by tokenizer. Each line in sequences has 11 words. Now we will split each line such that the first 10 words are in X and the last word is in y.

In [39]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]

array([ 364, 2035,  367,  366,  364,  365,  363,  217,  362,   19])

In [40]:
vocab_size = len(tokenizer.word_index) + 1

In [41]:
y = to_categorical(y, num_classes=vocab_size)

In [42]:
seq_length = X.shape[1]
seq_length

10

In [43]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 50)            101800    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2036)              205636    
Total params: 458,336
Trainable params: 458,336
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [45]:
model.fit(X, y, batch_size = 256, epochs = 100)

Epoch 1/100
16/16 [==============================] - 34s 11ms/step - loss: 7.6179 - accuracy: 0.0059
Epoch 2/100
16/16 [==============================] - 0s 12ms/step - loss: 7.5536 - accuracy: 0.0155
Epoch 3/100
16/16 [==============================] - 0s 10ms/step - loss: 7.1772 - accuracy: 0.0160
Epoch 4/100
16/16 [==============================] - 0s 10ms/step - loss: 6.9938 - accuracy: 0.0132
Epoch 5/100
16/16 [==============================] - 0s 9ms/step - loss: 6.8662 - accuracy: 0.0213
Epoch 6/100
16/16 [==============================] - 0s 9ms/step - loss: 6.6938 - accuracy: 0.0236
Epoch 7/100
16/16 [==============================] - 0s 9ms/step - loss: 6.5582 - accuracy: 0.0219
Epoch 8/100
16/16 [==============================] - 0s 9ms/step - loss: 6.4686 - accuracy: 0.0241
Epoch 9/100
16/16 [==============================] - 0s 9ms/step - loss: 6.4073 - accuracy: 0.0231
Epoch 10/100
16/16 [==============================] - 0s 9ms/step - loss: 6.2986 - accuracy: 0.0291
Epoc

In [46]:
seed_text=lines[12]
seed_text

'novu karipukayoothi erivirakaay meenaveyilodu koode vazhi palatheri paadamezhuthiya jeevakathayude ediloru lipiyaayi'

In [47]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [48]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 5)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'ere vijanatha choozhum irulilum aashayathu'